In [1]:
import pandas as pd
import os
import scipy.stats as stats

def add_hline(latex: str, index: int) -> str:
    """
    Adds a horizontal `index` lines before the last line of the table

    Args:
        latex: latex table
        index: index of horizontal line insertion (in lines)
    """
    lines = latex.splitlines()
    lines.insert(len(lines) + index - 6 - 3, r'\midrule')
    return '\n'.join(lines).replace('NaN', '')

In [2]:
carpetas = ['GUROBI','CPLEX','TSM','ESGH','LPDH', 'Experimentos v2']

In [3]:
gur = []
tsm = []
esgh = []
lpdh = []
cplex = []
versiones = []

for c in carpetas:
    archivos = os.listdir(c)
    for ar in archivos: 
        # print(sorted(ar.rstrip(".xlsx").split("_")))
        tiempo, solver, capacidad, nodos  = sorted(ar.rstrip(".xlsx").split("_"))
        tiempo = int(tiempo)
        capacidad = int(capacidad[1:])
        nodos = int(nodos[1:])
        datos = pd.read_excel(f"{c}/{ar}")
        datos["Q"] = capacidad
        datos["nodos"] = nodos
        datos["tiempos"] = tiempo
        if solver == "GUROBI":
            gur.append(datos)
        if solver == "CPLEX":
            cplex.append(datos)
        elif solver == "ATSM":
            tsm.append(datos)
        elif solver == "ESGH":
            esgh.append(datos)
        elif solver == "LPDH":
            lpdh.append(datos)
        elif solver in ["Exp1", "Exp2", "Exp3", "Exp4", "Exp5"]:
            datos["version"] = "3SM-exp" + solver[-1]
            versiones.append(datos)
        else: pass
        
gurobis = pd.concat(gur)
cplexs  = pd.concat(cplex)
tsms    = pd.concat(tsm)
esghs   = pd.concat(esgh)
lpdhs   = pd.concat(lpdh)
versiones   = pd.concat(versiones)

In [4]:
gurobis[["LB","UB"]] = gurobis[["LB","UB"]].round(2)
cplexs[["LB","UB"]] = cplexs[["LB","UB"]].round(2)
tsms[["min", "avg"]] = tsms[["min", "avg"]].round(2)
versiones[["min", "avg"]] = versiones[["min", "avg"]].round(2)

In [5]:
bks = pd.concat((
    tsms.groupby(["name", "Q", "nodos"]).agg(bks = ("min", "min")).reset_index(), 
    gurobis.groupby(["name", "Q", "nodos"]).agg(bks = ("UB", "min")).reset_index(),
    cplexs.groupby(["name", "Q", "nodos"]).agg(bks = ("UB", "min")).reset_index(),
    esghs.groupby(["name", "Q", "nodos"]).agg(bks = ("cost", "min")).reset_index(),
    lpdhs.groupby(["name", "Q", "nodos"]).agg(bks = ("cost", "min")).reset_index(),
    versiones.groupby(["name", "Q", "nodos"]).agg(bks = ("min", "min")).reset_index(),
))

bks = bks.groupby(["name", "Q", "nodos"]).agg(bks = ("bks", "min")).reset_index()

gurobis = gurobis.merge(bks, on = ["name", "Q", "nodos"])
cplexs  = cplexs.merge(bks, on = ["name", "Q", "nodos"] )
tsms    = tsms.merge(bks, on = ["name", "Q", "nodos"])
esghs   = esghs.merge(bks, on = ["name", "Q", "nodos"])
lpdhs   = lpdhs.merge(bks, on = ["name", "Q", "nodos"])
versiones = versiones.merge(bks, on = ["name", "Q", "nodos"])

In [6]:
gurobis["best"] = gurobis["UB"]
cplexs["best"] = cplexs["UB"]
tsms["best"] = tsms["min"]
versiones["best"] = versiones["min"]
lpdhs["best"] = lpdhs["cost"]
esghs["best"] = esghs["cost"]

gurobis["gap_best"]    = 100 * (gurobis["UB"] - gurobis["bks"]) / gurobis["bks"]
cplexs["gap_best"]     = 100 * (cplexs["UB"] - cplexs["bks"]) / cplexs["bks"]
tsms["gap_best"]       = 100 * (tsms["min"] - tsms["bks"]) / tsms["bks"]
tsms["gap_avg"]        = 100 * (tsms["avg"] - tsms["bks"]) / tsms["bks"]
versiones["gap_best"]  = 100 * (versiones["min"] - versiones["bks"]) / versiones["bks"]
versiones["gap_avg"]   = 100 * (versiones["avg"] - versiones["bks"]) / versiones["bks"]
lpdhs["gap"]           = 100 * (lpdhs["cost"] - lpdhs["bks"]) / lpdhs["bks"]
esghs["gap"]           = 100 * (esghs["cost"] - esghs["bks"]) / esghs["bks"]

gurobis["hit"]      = ( gurobis["best"]   - gurobis["bks"]   ).apply(lambda x: 1 if x == 0 else 0) 
tsms["hit"]         = ( tsms["best"]      - tsms["bks"]      ).apply(lambda x: 1 if x == 0 else 0) 
versiones["hit"]    = ( versiones["best"] - versiones["bks"] ).apply(lambda x: 1 if x == 0 else 0) 
lpdhs["hit"]        = ( lpdhs["best"]     - lpdhs["bks"]     ).apply(lambda x: 1 if x == 0 else 0) 
esghs["hit"]        = ( esghs["best"]     - esghs["bks"]     ).apply(lambda x: 1 if x == 0 else 0) 

gurobis = gurobis[gurobis.tiempos != 120]

# Tabla Comparación Solvers

In [7]:
gurobis3600 = gurobis[gurobis.tiempos == 3600]
cplexs3600 = cplexs[cplexs.tiempos == 3600]

gurobis3600["hit"] = gurobis3600.gap.apply(lambda x: 1 if x < 0.0001 else 0)
cplexs3600["hit"] = cplexs3600.gap.apply(lambda x: 1 if x < 0.0001 else 0)

C:\Users\rablo\AppData\Local\Temp\ipykernel_29656\1280445560.py:4: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  gurobis3600["hit"] = gurobis3600.gap.apply(lambda x: 1 if x < 0.0001 else 0)


In [8]:
gurobis3600

,name,LB,UB,gap,t,Q,nodos,tiempos,bks,best,gap_best,hit
2,rc2_2_10,1545.69,1651.44,0.064031,3600.007883,10,150,3600,1651.44,1651.44,0.000000,0
6,r2_2_6,1616.73,1693.25,0.045193,3600.018977,10,150,3600,1693.25,1693.25,0.000000,0
10,rc1_2_5,1632.05,1703.20,0.041777,3600.009419,10,150,3600,1703.20,1703.20,0.000000,0
14,c1_2_2,1261.87,1337.86,0.056802,3600.007489,10,150,3600,1337.86,1337.86,0.000000,0
18,r2_2_8,1484.46,1580.98,0.061048,3600.008291,10,150,3600,1579.55,1580.98,0.090532,0
...,...,...,...,...,...,...,...,...,...,...,...,...
2950,R108,799.64,813.09,0.016538,3600.002596,5,100,3600,813.09,813.09,0.000000,0
2952,RC108,935.02,957.51,0.023489,3600.003477,5,100,3600,957.51,957.51,0.000000,0
2954,R203,816.05,830.64,0.017576,3600.003606,5,100,3600,830.64,830.64,0.000000,0
2956,RC104,940.12,952.69,0.013194,3600.564430,5,100,3600,952.69,952.69,0.000000,0


In [9]:
t1_gurobi = gurobis3600[gurobis3600.nodos == 100].groupby(["Q"]).agg(
    Gap = ("gap_best", "mean"),
    LB_Gap = ("gap", "mean"),
    Hits = ("hit", "sum"),
    Time = ("t", "mean")).round(2)

t1_cplex = cplexs3600[cplexs3600.nodos == 100].groupby(["Q"]).agg(
    Gap = ("gap_best", "mean"),
    LB_Gap = ("gap", "mean"),
    Hits = ("hit", "sum"),
    Time = ("t", "mean")).round(2)

filas = ["\code{gap}$^\code{UB} (\%)", "\code{gap}$^\code{LB} (\%)", "hits", "time (s)"]
t1_gurobi.columns = ["Gurobi_" + f for f in filas]
t1_cplex.columns = ["CPLEX_" + f for f in filas]
t1 = pd.concat((t1_gurobi, t1_cplex), axis = 1)
t1.columns = pd.MultiIndex.from_tuples([c.split("_") for c in t1.columns])
t1 = t1.sort_values("Q", ascending = False)
t1.loc["overall"] = t1.mean().round(2)
#t1.loc["overall"]["Gurobi","Hits"] = 0
#t1.loc["overall"]["CPLEX","Hits"] = 0
#t1.loc["overall"]["Gurobi","Hits"] = t1["Gurobi","Hits"].sum()
#t1.loc["overall"]["CPLEX","Hits"]  = t1["CPLEX","Hits"].sum()
t1.reset_index(inplace = True)
t1.loc[t1["Q"] == 1000, "Q"] = "$\infty$"
t1

Q                     Gurobi                                   \
            \code{gap}$^\code{UB} (\%) \code{gap}$^\code{LB} (\%)  hits   
0  $\infty$                       0.00                       0.00  55.0   
1        20                       0.00                       0.01  39.0   
2        15                       0.02                       0.01  28.0   
3        10                       0.01                       0.02  13.0   
4         5                       0.01                       0.01   9.0   
5   overall                       0.01                       0.01  28.8   

                                CPLEX                                   \
  time (s) \code{gap}$^\code{UB} (\%) \code{gap}$^\code{LB} (\%)  hits   
0   261.94                       0.03                       0.01  35.0   
1  1492.59                       0.16                       0.02  16.0   
2  2317.87                       0.26                       0.03   9.0   
3  2962.13                       0.01                       0.02  13.0   
4  3200.20                       0.41                       0.03   1.0   
5  2046.95                       0.17                       0.02  14.8   

            
  time (s)  
0  1698.18  
1  2818.16  
2  3202.71  
3  2962.13  
4  3536.29  
5  2843.49

In [10]:
latex1 = t1.to_latex(
    index = False,
    column_format = "c rrrr rrrr",
    multicolumn_format = "c",
    multirow = True, 
    caption =  "Performance of comercial solvers",
    label = "tab:solvers_results",
    position = "H",
    float_format='%.2f',
    escape = False)

text_latex1 = add_hline(latex1, index = 5)
with open("latex/solver_results.tex", "a") as file:
    file.write(text_latex1)
    
print(text_latex1)

\begin{table}[H]
\caption{Performance of comercial solvers}
\label{tab:solvers_results}
\begin{tabular}{c rrrr rrrr}
\toprule
Q & \multicolumn{4}{c}{Gurobi} & \multicolumn{4}{c}{CPLEX} \\
 & \code{gap}$^\code{UB} (\%) & \code{gap}$^\code{LB} (\%) & hits & time (s) & \code{gap}$^\code{UB} (\%) & \code{gap}$^\code{LB} (\%) & hits & time (s) \\
\midrule
$\infty$ & 0.00 & 0.00 & 55.00 & 261.94 & 0.03 & 0.01 & 35.00 & 1698.18 \\
20 & 0.00 & 0.01 & 39.00 & 1492.59 & 0.16 & 0.02 & 16.00 & 2818.16 \\
15 & 0.02 & 0.01 & 28.00 & 2317.87 & 0.26 & 0.03 & 9.00 & 3202.71 \\
10 & 0.01 & 0.02 & 13.00 & 2962.13 & 0.01 & 0.02 & 13.00 & 2962.13 \\
5 & 0.01 & 0.01 & 9.00 & 3200.20 & 0.41 & 0.03 & 1.00 & 3536.29 \\
\midrule
overall & 0.01 & 0.01 & 28.80 & 2046.95 & 0.17 & 0.02 & 14.80 & 2843.49 \\
\bottomrule
\end{tabular}
\end{table}


# Tablas Griegos

In [11]:
tESGH = esghs[esghs.nodos == 100].groupby(["Q"]).agg(
    Best = ("best", "mean"),
    Gap  = ("gap", "mean"),
    Hits = ("hit", "sum")).round(2)

tTSM  = tsms[(tsms.nodos == 100) & (tsms.tiempos == 60)].groupby(["Q"]).agg(
    Best = ("best", "mean"),
    Avg  = ("avg", "mean"),
    Min_gap = ("gap_best", "mean"),
    Avg_gap = ("gap_avg", "mean"),
    Hits = ("hit", "sum")).round(2)

tESGH.columns = ["ESGH_" + f for f in ["best","\code{gap}$^\code{best} (\%)", "hits"]]
tTSM.columns  = ["3SM (60s)_" + f for f in ["best", "avg", "\code{gap}$^\code{best} (\%)","\code{gap}$^\code{avg} (\%)", "hits"]]
t2 = pd.concat((tESGH, tTSM), axis = 1)
t2.columns = pd.MultiIndex.from_tuples([c.split("_") for c in t2.columns])
t2 = t2.sort_values("Q", ascending = False)
t2["Improvement (\%)", "best"] = 100 * (t2["ESGH", "best"] - t2["3SM (60s)", "best"]) / t2["ESGH", "best"]
t2["Improvement (\%)", "avg"] = 100 * (t2["ESGH", "best"] - t2["3SM (60s)", "avg"]) / t2["ESGH", "best"]
t2["Improvement (\%)", "best"] = t2["Improvement (\%)", "best"].round(2)
t2["Improvement (\%)", "avg"] = t2["Improvement (\%)", "avg"].round(2)
t2.loc["overall"] = t2.mean().round(2)
t2.reset_index(inplace = True)
t2.loc[t2["Q"] == 1000, "Q"] = "$\infty$"
t2 = t2.drop([("ESGH", "best"),("3SM (60s)","best"), ("3SM (60s)","avg")], axis = 1)
t2

Q                         ESGH                         3SM (60s)  \
            \code{gap}$^\code{best} (\%) hits \code{gap}$^\code{best} (\%)   
0  $\infty$                         8.69  2.0                         0.07   
1        20                        18.00  0.0                         0.16   
2        15                        18.91  0.0                         0.33   
3        10                        16.37  0.0                         0.67   
4         5                        13.03  0.0                         1.17   
5   overall                        15.00  0.4                         0.48   

                                    Improvement (\%)         
  \code{gap}$^\code{avg} (\%)  hits             best    avg  
0                        0.20  40.0             8.15   8.03  
1                        0.34  35.0            15.20  15.05  
2                        0.67  21.0            15.77  15.49  
3                        1.21  10.0            13.56  13.09  
4                        1.81   6.0            10.47   9.89  
5                        0.85  22.4            12.63  12.31

In [12]:
latex2 = t2.to_latex(
    index = False,
    column_format = "c rr rrr rr",
    multicolumn_format = "c",
    multirow = True, 
    caption =  "Performance against ESGH",
    label = "tab:3lm_resuts_kritikos1",
    position = "H",
    float_format='%.2f',
    escape = False)

text_latex2 = add_hline(latex2, index = 5)

with open("latex/3LM100 - ESGH.tex", "a") as file:
    file.write(text_latex2)
print(text_latex2)

\begin{table}[H]
\caption{Performance against ESGH}
\label{tab:3lm_resuts_kritikos1}
\begin{tabular}{c rr rrr rr}
\toprule
Q & \multicolumn{2}{c}{ESGH} & \multicolumn{3}{c}{3SM (60s)} & \multicolumn{2}{c}{Improvement (\%)} \\
 & \code{gap}$^\code{best} (\%) & hits & \code{gap}$^\code{best} (\%) & \code{gap}$^\code{avg} (\%) & hits & best & avg \\
\midrule
$\infty$ & 8.69 & 2.00 & 0.07 & 0.20 & 40.00 & 8.15 & 8.03 \\
20 & 18.00 & 0.00 & 0.16 & 0.34 & 35.00 & 15.20 & 15.05 \\
15 & 18.91 & 0.00 & 0.33 & 0.67 & 21.00 & 15.77 & 15.49 \\
10 & 16.37 & 0.00 & 0.67 & 1.21 & 10.00 & 13.56 & 13.09 \\
5 & 13.03 & 0.00 & 1.17 & 1.81 & 6.00 & 10.47 & 9.89 \\
\midrule
overall & 15.00 & 0.40 & 0.48 & 0.85 & 22.40 & 12.63 & 12.31 \\
\bottomrule
\end{tabular}
\end{table}


In [13]:
tESGH = esghs[(esghs.nodos == 100) & (esghs.name.str.contains("R1")) ].groupby(["Q"]).agg(
    Best = ("best", "mean"),
    Gap  = ("gap", "mean"),
    Hits = ("hit", "sum")).round(2)

tLPDH = lpdhs[(lpdhs.nodos == 100) & (lpdhs.name.str.contains("R1")) ].groupby(["Q"]).agg(
    Best = ("best", "mean"),
    Gap  = ("gap", "mean"),
    Hits = ("hit", "sum")).round(2)

tTSM  = tsms[(tsms.nodos == 100) & (tsms.tiempos == 60) & (tsms.name.str.contains("R1"))].groupby(["Q"]).agg(
    Best = ("best", "mean"),
    Avg  = ("avg", "mean"),
    Min_gap = ("gap_best", "mean"),
    Avg_gap = ("gap_avg", "mean"),
    Hits = ("hit", "sum")).round(2)

tESGH.columns = ["ESGH_" + f for f in ["best","\code{gap}$^\code{best} (\%)", "hits"]]
tLPDH.columns = ["LPDH_" + f for f in ["best","\code{gap}$^\code{best} (\%)", "hits"]]
tTSM.columns  = ["3SM (60s)_" + f for f in ["best", "avg", "\code{gap}$^\code{best} (\%)","\code{gap}$^\code{avg} (\%)", "hits"]]
t3 = pd.concat((tESGH, tLPDH, tTSM), axis = 1)
t3.columns = pd.MultiIndex.from_tuples([c.split("_") for c in t3.columns])
t3 = t3.sort_values("Q", ascending = False)
t3["Improvement (\%)", "best"] = 100 * (t3["LPDH", "best"] - t3["3SM (60s)", "best"]) / t3["LPDH", "best"]
t3["Improvement (\%)", "avg"] = 100 * (t3["LPDH", "best"] - t3["3SM (60s)", "avg"]) / t3["LPDH", "best"]
t3["Improvement (\%)", "best"] = t3["Improvement (\%)", "best"].round(2)
t3["Improvement (\%)", "avg"] = t3["Improvement (\%)", "avg"].round(2)
t3.loc["overall"] = t3.mean().round(2)
t3.reset_index(inplace = True)
t3.loc[t3["Q"] == 1000, "Q"] = "$\infty$"
t3 = t3.drop([("ESGH", "best"),("LPDH", "best"),("3SM (60s)","best"), ("3SM (60s)","avg")], axis = 1)
t3

Q                         ESGH                              LPDH  \
            \code{gap}$^\code{best} (\%) hits \code{gap}$^\code{best} (\%)   
0  $\infty$                         9.20  1.0                         7.95   
1        20                        15.82  0.0                        13.51   
2        15                        15.91  0.0                        13.24   
3        10                        13.86  0.0                        12.23   
4         5                        13.21  0.0                        11.63   
5   overall                        13.60  0.2                        11.71   

                          3SM (60s)                                   \
  hits \code{gap}$^\code{best} (\%) \code{gap}$^\code{avg} (\%) hits   
0  1.0                         0.06                        0.17  5.0   
1  0.0                         0.18                        0.47  6.0   
2  0.0                         0.33                        0.74  3.0   
3  0.0                         0.42                        0.65  1.0   
4  0.0                         0.78                        1.22  2.0   
5  0.2                         0.35                        0.65  3.4   

  Improvement (\%)         
              best    avg  
0             7.28   7.18  
1            11.64  11.38  
2            11.31  10.96  
3            10.53  10.33  
4             9.81   9.43  
5            10.11   9.86

In [14]:
latex3 = t3.to_latex(
    index = False,
    column_format = "c rr rr rrr rr",
    multicolumn_format = "c",
    multirow = True, 
    caption =  "Performance against LPDH",
    label = "tab:3lm_resuts_kritikos2",
    position = "H",
    float_format='%.2f',
    escape = False)

text_latex3 = add_hline(latex3, index = 5)

with open("latex/3LM100 - LPDH.tex", "a") as file:
    file.write(text_latex3)
print(text_latex3)

\begin{table}[H]
\caption{Performance against LPDH}
\label{tab:3lm_resuts_kritikos2}
\begin{tabular}{c rr rr rrr rr}
\toprule
Q & \multicolumn{2}{c}{ESGH} & \multicolumn{2}{c}{LPDH} & \multicolumn{3}{c}{3SM (60s)} & \multicolumn{2}{c}{Improvement (\%)} \\
 & \code{gap}$^\code{best} (\%) & hits & \code{gap}$^\code{best} (\%) & hits & \code{gap}$^\code{best} (\%) & \code{gap}$^\code{avg} (\%) & hits & best & avg \\
\midrule
$\infty$ & 9.20 & 1.00 & 7.95 & 1.00 & 0.06 & 0.17 & 5.00 & 7.28 & 7.18 \\
20 & 15.82 & 0.00 & 13.51 & 0.00 & 0.18 & 0.47 & 6.00 & 11.64 & 11.38 \\
15 & 15.91 & 0.00 & 13.24 & 0.00 & 0.33 & 0.74 & 3.00 & 11.31 & 10.96 \\
10 & 13.86 & 0.00 & 12.23 & 0.00 & 0.42 & 0.65 & 1.00 & 10.53 & 10.33 \\
5 & 13.21 & 0.00 & 11.63 & 0.00 & 0.78 & 1.22 & 2.00 & 9.81 & 9.43 \\
\midrule
overall & 13.60 & 0.20 & 11.71 & 0.20 & 0.35 & 0.65 & 3.40 & 10.11 & 9.86 \\
\bottomrule
\end{tabular}
\end{table}


# Tablas comparativas con gurobi

In [15]:
def add_hline2(latex: str, index: int) -> str:
    """
    Adds a horizontal `index` lines before the last line of the table

    Args:
        latex: latex table
        index: index of horizontal line insertion (in lines)
    """
    lines = latex.splitlines()
    lines[7] = r"\cmidrule(lr){1-1} \cmidrule(lr){2-5} \cmidrule(lr){2-5} \cmidrule(lr){6-7} \cmidrule(lr){8-10} \cmidrule(lr){11-12}"  
    lines.insert(len(lines) + index - 6 - 3, r'\midrule')
    return '\n'.join(lines).replace('NaN', '')
    
def hacer_tabla(n,t):
    tGurobi3600 = gurobis3600[gurobis3600.nodos == n].groupby(["Q"]).agg(
        Gap    = ("gap_best", "mean"),
        LB_Gap = ("gap", "mean"),
        Time   = ("t", "mean"),
        Hits = ("hit", "sum")).round(2)

    tGurobi = gurobis[(gurobis.nodos == n) & (gurobis.tiempos == t) ].groupby(["Q"]).agg(
        Best = ("best", "mean"),
        Gap  = ("gap_best", "mean"),
        Hits = ("hit", "sum")).round(2)

    tTSM  = tsms[(tsms.nodos == n) & (tsms.tiempos == t)].groupby(["Q"]).agg(
        Best = ("best", "mean"),
        Avg  = ("avg", "mean"),
        Min_gap = ("gap_best", "mean"),
        Avg_gap = ("gap_avg", "mean"),
        Hits = ("hit", "sum")).round(2)
    
    tGurobi3600.columns = ["Gurobi (3600s)_" + f for f in ["\code{gap}$^\code{UB} (\%)", "\code{gap}$^\code{LM} (\%)", "time", "hits"]]
    tGurobi.columns = [f"Gurobi ({t}s)_" + f for f in ["best","\code{gap}$^\code{UB} (\%)", "hits"]]
    tTSM.columns  = [f"3SM ({t}s)_" + f for f in ["best", "avg", "\code{gap}$^\code{best} (\%)","\code{gap}$^\code{avg} (\%)", "hits"]]
    table = pd.concat((tGurobi3600, tGurobi, tTSM), axis = 1)
    table.columns = pd.MultiIndex.from_tuples([c.split("_") for c in table.columns])
    table = table.sort_values("Q", ascending = False)
    table["Improvement (\%)", "best"] = 100 * (table[f"Gurobi ({t}s)", "best"] - table[f"3SM ({t}s)", "best"]) / table[f"Gurobi ({t}s)", "best"]
    table["Improvement (\%)", "avg"] = 100 * (table[f"Gurobi ({t}s)", "best"] - table[f"3SM ({t}s)", "avg"]) / table[f"Gurobi ({t}s)", "best"]
    table["Improvement (\%)", "best"] = table["Improvement (\%)", "best"].round(2)
    table["Improvement (\%)", "avg"] = table["Improvement (\%)", "avg"].round(2)
    table.loc["overall"] = table.mean().round(2)
    table.reset_index(inplace = True)
    table.loc[table["Q"] == 1000, "Q"] = "$\infty$"
    table = table.drop([(f"Gurobi ({t}s)","best"), (f"3SM ({t}s)", "best"), (f"3SM ({t}s)", "avg")], axis = 1)
    size = size = "small" if n == 100 else "medium" if n == 150 else "large"
    latex = table.to_latex(
        index = False,
        column_format = "c rrrr rr rrr rr",
        multicolumn_format = "c",
        multirow = True, 
        caption =  f"Performance against Gurobi on {size} instances in {t} seconds",
        label = f"tab:3lm_resuts{n}T{t}",
        position = "H",
        float_format='%.2f',
        escape = False)
    
    text_latex = add_hline2(latex, index = 5)
    with open(f"latex/3SM{n}T{t}.tex", "a") as file:
        file.write(text_latex)
    
    return text_latex

In [16]:
print(hacer_tabla(100,60))

\begin{table}[H]
\caption{Performance against Gurobi on small instances in 60 seconds}
\label{tab:3lm_resuts100T60}
\begin{tabular}{c rrrr rr rrr rr}
\toprule
Q & \multicolumn{4}{c}{Gurobi (3600s)} & \multicolumn{2}{c}{Gurobi (60s)} & \multicolumn{3}{c}{3SM (60s)} & \multicolumn{2}{c}{Improvement (\%)} \\
 & \code{gap}$^\code{UB} (\%) & \code{gap}$^\code{LM} (\%) & time & hits & \code{gap}$^\code{UB} (\%) & hits & \code{gap}$^\code{best} (\%) & \code{gap}$^\code{avg} (\%) & hits & best & avg \\
\cmidrule(lr){1-1} \cmidrule(lr){2-5} \cmidrule(lr){2-5} \cmidrule(lr){6-7} \cmidrule(lr){8-10} \cmidrule(lr){11-12}
$\infty$ & 0.00 & 0.00 & 261.94 & 55.00 & 0.14 & 41.00 & 0.07 & 0.20 & 40.00 & 0.07 & -0.07 \\
20 & 0.00 & 0.01 & 1492.59 & 39.00 & 0.47 & 23.00 & 0.16 & 0.34 & 35.00 & 0.31 & 0.13 \\
15 & 0.02 & 0.01 & 2317.87 & 28.00 & 0.57 & 19.00 & 0.33 & 0.67 & 21.00 & 0.25 & -0.08 \\
10 & 0.01 & 0.02 & 2962.13 & 13.00 & 1.06 & 11.00 & 0.67 & 1.21 & 10.00 & 0.39 & -0.15 \\
5 & 0.01 & 0.01 & 3

In [17]:
print(hacer_tabla(150,60))

\begin{table}[H]
\caption{Performance against Gurobi on medium instances in 60 seconds}
\label{tab:3lm_resuts150T60}
\begin{tabular}{c rrrr rr rrr rr}
\toprule
Q & \multicolumn{4}{c}{Gurobi (3600s)} & \multicolumn{2}{c}{Gurobi (60s)} & \multicolumn{3}{c}{3SM (60s)} & \multicolumn{2}{c}{Improvement (\%)} \\
 & \code{gap}$^\code{UB} (\%) & \code{gap}$^\code{LM} (\%) & time & hits & \code{gap}$^\code{UB} (\%) & hits & \code{gap}$^\code{best} (\%) & \code{gap}$^\code{avg} (\%) & hits & best & avg \\
\cmidrule(lr){1-1} \cmidrule(lr){2-5} \cmidrule(lr){2-5} \cmidrule(lr){6-7} \cmidrule(lr){8-10} \cmidrule(lr){11-12}
$\infty$ & 0.00 & 0.00 & 746.76 & 53.00 & 0.61 & 26.00 & 1.12 & 1.91 & 27.00 & -0.45 & -1.19 \\
20 & 0.08 & 0.03 & 3255.56 & 8.00 & 1.30 & 8.00 & 0.81 & 1.54 & 9.00 & 0.48 & -0.24 \\
15 & 0.08 & 0.03 & 3412.33 & 4.00 & 1.87 & 4.00 & 1.12 & 2.14 & 4.00 & 0.67 & -0.30 \\
10 & 0.09 & 0.04 & 3502.47 & 2.00 & 2.36 & 3.00 & 1.73 & 2.79 & 1.00 & 0.62 & -0.39 \\
5 & 0.00 & 0.02 & 3549.49

In [18]:
print(hacer_tabla(200,60))

\begin{table}[H]
\caption{Performance against Gurobi on large instances in 60 seconds}
\label{tab:3lm_resuts200T60}
\begin{tabular}{c rrrr rr rrr rr}
\toprule
Q & \multicolumn{4}{c}{Gurobi (3600s)} & \multicolumn{2}{c}{Gurobi (60s)} & \multicolumn{3}{c}{3SM (60s)} & \multicolumn{2}{c}{Improvement (\%)} \\
 & \code{gap}$^\code{UB} (\%) & \code{gap}$^\code{LM} (\%) & time & hits & \code{gap}$^\code{UB} (\%) & hits & \code{gap}$^\code{best} (\%) & \code{gap}$^\code{avg} (\%) & hits & best & avg \\
\cmidrule(lr){1-1} \cmidrule(lr){2-5} \cmidrule(lr){2-5} \cmidrule(lr){6-7} \cmidrule(lr){8-10} \cmidrule(lr){11-12}
$\infty$ & 0.00 & 0.00 & 1650.73 & 39.00 & 20.02 & 11.00 & 4.18 & 6.27 & 8.00 & 12.10 & 10.39 \\
20 & 0.07 & 0.04 & 3552.94 & 1.00 & 12.05 & 0.00 & 3.27 & 5.65 & 0.00 & 7.40 & 5.34 \\
15 & 0.13 & 0.05 & 3586.08 & 1.00 & 9.16 & 0.00 & 3.59 & 6.06 & 0.00 & 4.82 & 2.62 \\
10 & 0.05 & 0.06 & 3600.38 & 0.00 & 9.06 & 1.00 & 4.48 & 7.07 & 0.00 & 4.01 & 1.66 \\
5 & 0.02 & 0.04 & 3600.24 &

In [19]:
print(hacer_tabla(150,180))

\begin{table}[H]
\caption{Performance against Gurobi on medium instances in 180 seconds}
\label{tab:3lm_resuts150T180}
\begin{tabular}{c rrrr rr rrr rr}
\toprule
Q & \multicolumn{4}{c}{Gurobi (3600s)} & \multicolumn{2}{c}{Gurobi (180s)} & \multicolumn{3}{c}{3SM (180s)} & \multicolumn{2}{c}{Improvement (\%)} \\
 & \code{gap}$^\code{UB} (\%) & \code{gap}$^\code{LM} (\%) & time & hits & \code{gap}$^\code{UB} (\%) & hits & \code{gap}$^\code{best} (\%) & \code{gap}$^\code{avg} (\%) & hits & best & avg \\
\cmidrule(lr){1-1} \cmidrule(lr){2-5} \cmidrule(lr){2-5} \cmidrule(lr){6-7} \cmidrule(lr){8-10} \cmidrule(lr){11-12}
$\infty$ & 0.00 & 0.00 & 746.76 & 53.00 & 0.45 & 25.00 & 0.06 & 0.22 & 42.00 & 0.39 & 0.24 \\
20 & 0.08 & 0.03 & 3255.56 & 8.00 & 1.68 & 10.00 & 0.40 & 0.79 & 18.00 & 1.23 & 0.85 \\
15 & 0.08 & 0.03 & 3412.33 & 4.00 & 2.07 & 2.00 & 0.62 & 1.12 & 9.00 & 1.36 & 0.89 \\
10 & 0.09 & 0.04 & 3502.47 & 2.00 & 2.16 & 2.00 & 0.73 & 1.24 & 4.00 & 1.39 & 0.90 \\
5 & 0.00 & 0.02 & 3549.4

In [20]:
print(hacer_tabla(200,180))

\begin{table}[H]
\caption{Performance against Gurobi on large instances in 180 seconds}
\label{tab:3lm_resuts200T180}
\begin{tabular}{c rrrr rr rrr rr}
\toprule
Q & \multicolumn{4}{c}{Gurobi (3600s)} & \multicolumn{2}{c}{Gurobi (180s)} & \multicolumn{3}{c}{3SM (180s)} & \multicolumn{2}{c}{Improvement (\%)} \\
 & \code{gap}$^\code{UB} (\%) & \code{gap}$^\code{LM} (\%) & time & hits & \code{gap}$^\code{UB} (\%) & hits & \code{gap}$^\code{best} (\%) & \code{gap}$^\code{avg} (\%) & hits & best & avg \\
\cmidrule(lr){1-1} \cmidrule(lr){2-5} \cmidrule(lr){2-5} \cmidrule(lr){6-7} \cmidrule(lr){8-10} \cmidrule(lr){11-12}
$\infty$ & 0.00 & 0.00 & 1650.73 & 39.00 & 9.69 & 14.00 & 1.02 & 1.80 & 23.00 & 7.39 & 6.72 \\
20 & 0.07 & 0.04 & 3552.94 & 1.00 & 6.19 & 0.00 & 0.98 & 1.81 & 3.00 & 4.70 & 3.96 \\
15 & 0.13 & 0.05 & 3586.08 & 1.00 & 3.67 & 1.00 & 1.14 & 2.02 & 10.00 & 2.35 & 1.52 \\
10 & 0.05 & 0.06 & 3600.38 & 0.00 & 3.28 & 0.00 & 1.48 & 2.23 & 0.00 & 1.71 & 1.00 \\
5 & 0.02 & 0.04 & 3600.24

# Comparación Experimentos

In [21]:
tsms_versiones = tsms[(tsms.tiempos == 60) & (tsms.nodos == 100)].copy()
exp_versiones = versiones[(versiones.tiempos == 60) & (versiones.nodos == 100)].copy()
tsms_versiones.loc[:,"version"] = "3SM"

# gurobis_versiones = gurobis[(gurobis.tiempos == 60) & (gurobis.nodos == 100)].copy()
# gurobis_versiones.loc[:,"version"] = "gurobi60"
# gurobis_versiones.loc[:,"t_avg"]   = gurobis_versiones.loc[:,"t"]
# gurobis_versiones.loc[:,"gap_avg"]   = gurobis_versiones.loc[:,"gap_best"]

versiones2 = pd.concat((exp_versiones , tsms_versiones))
tabla_versiones = versiones2.groupby(["version"]).agg(gap_best  = ("gap_best", "mean"),
                                   gap_avg  =  ("gap_avg", "mean"),
                                   avg_time =  ("t_avg", "mean"),
                                   hit_sum  =  ("hit", "sum")).reset_index()

# tabla_versiones.sort_values(["nodos","version"]).set_index(["nodos", "version"])
tabla_versiones[["gap_best", "gap_avg"]] = tabla_versiones[["gap_best", "gap_avg"]] 
tabla_versiones.columns = ["Experiment", "\code{gap}$^\code{best} (\%)", "\code{gap}$^\code{avg} (\%)", "time (avg)", "hits"]
tabla_versiones

,Experiment,\code{gap}$^\code{best} (\%),\code{gap}$^\code{avg} (\%),time (avg),hits
0,3SM,0.481058,0.845812,54.755474,112
1,3SM-exp1,2.383065,4.229000,60.005359,11
2,3SM-exp2,1.223105,2.022402,57.116093,59
3,3SM-exp3,0.865756,1.526353,57.143192,70
4,3SM-exp4,0.953122,1.661638,57.136239,75
5,3SM-exp5,0.488166,0.853755,54.617926,113


In [22]:
tabla_versiones_latex = tabla_versiones.to_latex(
    index = False,
    column_format = "c rrrr",
    multicolumn_format = "c",
    multirow = True, 
    caption =  "Experiments for the performance of matheuristic operators",
    label = "tab:matheuristic_experiments",
    position = "H",
    float_format='%.2f',
    escape = False)

print(tabla_versiones_latex)

\begin{table}[H]
\caption{Experiments for the performance of matheuristic operators}
\label{tab:matheuristic_experiments}
\begin{tabular}{c rrrr}
\toprule
Experiment & \code{gap}$^\code{best} (\%) & \code{gap}$^\code{avg} (\%) & time (avg) & hits \\
\midrule
3SM & 0.48 & 0.85 & 54.76 & 112 \\
3SM-exp1 & 2.38 & 4.23 & 60.01 & 11 \\
3SM-exp2 & 1.22 & 2.02 & 57.12 & 59 \\
3SM-exp3 & 0.87 & 1.53 & 57.14 & 70 \\
3SM-exp4 & 0.95 & 1.66 & 57.14 & 75 \\
3SM-exp5 & 0.49 & 0.85 & 54.62 & 113 \\
\bottomrule
\end{tabular}
\end{table}



In [23]:
tsms_versiones = tsms[(tsms.tiempos == 180) & (tsms.nodos >= 150)].copy()
tsms_versiones.loc[:,"version"] = "3SM"
exp_versiones = versiones[(versiones.tiempos == 180) & (versiones.nodos >= 150)].copy()

# gurobis_versiones = gurobis[(gurobis.tiempos == 180) & (gurobis.nodos >= 150)].copy()
# gurobis_versiones.loc[:,"version"] = "gurobi60"
# gurobis_versiones.loc[:,"t_avg"]   = gurobis_versiones.loc[:,"t"]
# gurobis_versiones.loc[:,"gap_avg"]   = gurobis_versiones.loc[:,"gap_best"]

versiones2 = pd.concat((exp_versiones , tsms_versiones))
tabla_versiones = versiones2.groupby(["version"]).agg(gap_best  = ("gap_best", "mean"),
                                   gap_avg  =  ("gap_avg", "mean"),
                                   avg_time =  ("t_avg", "mean"),
                                   hit_sum  =  ("hit", "sum")).reset_index()

# tabla_versiones.sort_values(["nodos","version"]).set_index(["nodos", "version"])
tabla_versiones[["gap_best", "gap_avg"]] = tabla_versiones[["gap_best", "gap_avg"]]
tabla_versiones.columns = ["Experiment", "\code{gap}$^\code{best} (\%)", "\code{gap}$^\code{avg} (\%)", "time (avg)", "hits"]
tabla_versiones

,Experiment,\code{gap}$^\code{best} (\%),\code{gap}$^\code{avg} (\%),time (avg),hits
0,3SM,0.960184,1.590728,173.468092,110
1,3SM-exp1,4.797266,7.155029,180.003799,0
2,3SM-exp2,1.511098,2.255302,173.668693,87
3,3SM-exp3,1.158434,1.877952,173.678090,111
4,3SM-exp4,1.012049,1.651786,173.671315,101
5,3SM-exp5,1.096095,1.767055,173.671834,106


In [24]:
tabla_versiones_latex = tabla_versiones.to_latex(
    index = False,
    column_format = "c rrrr",
    multicolumn_format = "c",
    multirow = True, 
    caption =  "Experiments for the performance of matheuristic operators",
    label = "tab:matheuristic_experiments",
    position = "H",
    float_format='%.2f',
    escape = False)

print(tabla_versiones_latex)

\begin{table}[H]
\caption{Experiments for the performance of matheuristic operators}
\label{tab:matheuristic_experiments}
\begin{tabular}{c rrrr}
\toprule
Experiment & \code{gap}$^\code{best} (\%) & \code{gap}$^\code{avg} (\%) & time (avg) & hits \\
\midrule
3SM & 0.96 & 1.59 & 173.47 & 110 \\
3SM-exp1 & 4.80 & 7.16 & 180.00 & 0 \\
3SM-exp2 & 1.51 & 2.26 & 173.67 & 87 \\
3SM-exp3 & 1.16 & 1.88 & 173.68 & 111 \\
3SM-exp4 & 1.01 & 1.65 & 173.67 & 101 \\
3SM-exp5 & 1.10 & 1.77 & 173.67 & 106 \\
\bottomrule
\end{tabular}
\end{table}



In [25]:
tsms_versiones = tsms[((tsms.tiempos == 180) & (tsms.nodos >= 150)) | ((tsms.tiempos == 60) & (tsms.nodos == 100))].copy()
tsms_versiones.loc[:,"version"] = "3SM"
exp_versiones = versiones[((versiones.tiempos == 180) & (versiones.nodos >= 150)) | ((versiones.tiempos == 60) & (versiones.nodos == 100))].copy()

# gurobis_versiones = gurobis[((gurobis.tiempos == 180) & (gurobis.nodos >= 150)) | ((gurobis.tiempos == 60) & (gurobis.nodos == 100))].copy()
# gurobis_versiones.loc[:,"version"] = "gurobi60"
# gurobis_versiones.loc[:,"t_avg"]   = gurobis_versiones.loc[:,"t"]
# gurobis_versiones.loc[:,"gap_avg"]   = gurobis_versiones.loc[:,"gap_best"]

versiones2 = pd.concat((exp_versiones , tsms_versiones))
tabla_versiones = versiones2.groupby(["version"]).agg(gap_best  = ("gap_best", "mean"),
                                   gap_avg  =  ("gap_avg", "mean"),
                                   avg_time =  ("t_avg", "mean"),
                                   hit_sum  =  ("hit", "sum")).reset_index()

# tabla_versiones.sort_values(["nodos","version"]).set_index(["nodos", "version"])#
tabla_versiones[["gap_best", "gap_avg"]] = tabla_versiones[["gap_best", "gap_avg"]]
tabla_versiones.columns = ["Experiment", "\code{gap}$^\code{best} (\%)", "\code{gap}$^\code{avg} (\%)", "time (avg)", "hits"]
tabla_versiones

,Experiment,\code{gap}$^\code{best} (\%),\code{gap}$^\code{avg} (\%),time (avg),hits
0,3SM,0.807735,1.353709,135.695895,222
1,3SM-exp1,4.029111,6.224020,141.822477,11
2,3SM-exp2,1.419464,2.181197,136.583775,146
3,3SM-exp3,1.065309,1.766080,136.598804,181
4,3SM-exp4,0.993300,1.654921,136.591973,176
5,3SM-exp5,0.902663,1.476460,135.791045,219


In [26]:
tabla_versiones_latex = tabla_versiones.to_latex(
    index = False,
    column_format = "c rrrr",
    multicolumn_format = "c",
    multirow = True, 
    caption =  "Experiments for the performance of matheuristic operators",
    label = "tab:matheuristic_experiments",
    position = "H",
    float_format='%.2f',
    escape = False)

print(tabla_versiones_latex)

\begin{table}[H]
\caption{Experiments for the performance of matheuristic operators}
\label{tab:matheuristic_experiments}
\begin{tabular}{c rrrr}
\toprule
Experiment & \code{gap}$^\code{best} (\%) & \code{gap}$^\code{avg} (\%) & time (avg) & hits \\
\midrule
3SM & 0.81 & 1.35 & 135.70 & 222 \\
3SM-exp1 & 4.03 & 6.22 & 141.82 & 11 \\
3SM-exp2 & 1.42 & 2.18 & 136.58 & 146 \\
3SM-exp3 & 1.07 & 1.77 & 136.60 & 181 \\
3SM-exp4 & 0.99 & 1.65 & 136.59 & 176 \\
3SM-exp5 & 0.90 & 1.48 & 135.79 & 219 \\
\bottomrule
\end{tabular}
\end{table}



# Wilcoxon

In [27]:
def wilcoxon(gurobis, tsms, base = "gap_best", 
             tiempos = [60, 180], nodos = [100, 150, 200]):
    global exp1, exp2, exp3
    tabla = []
    for n in nodos:
        fila = dict()
        fila["tamaño"] = n
        for t in tiempos:
            exp1 = tsms[(tsms.tiempos == t) & (tsms.nodos == n)][base]
            exp2 = gurobis[(gurobis.tiempos == t) & (gurobis.nodos == n)][base]
            fila[f"gurobi {t}"] = stats.wilcoxon(exp1, exp2, alternative = "less").pvalue
            try:
                exp3 = gurobis[(gurobis.tiempos == 3600) & (gurobis.nodos == n)][base]
                fila[f"gurobi benckmark-{t}"] = stats.wilcoxon(exp1, exp3, alternative = "less").pvalue
            except: pass
        tabla.append(fila)
    return pd.DataFrame(tabla)
    
tabla = wilcoxon(gurobis, tsms)
tabla = tabla[sorted(tabla.columns)]

In [28]:
tabla2 = tabla.set_index("tamaño")
tabla2

,gurobi 180,gurobi 60,gurobi benckmark-180,gurobi benckmark-60
tamaño,,,,
100,NaN,6.816269e-06,NaN,1.0
150,4.366749e-31,1.221897e-07,1.0,1.0
200,4.928053e-33,1.449382e-13,1.0,1.0


In [35]:
def wilcoxon2(gurobis, tsms, base = "gap_best", 
             tiempos = [60, 180], nodos = [100, 150, 200]):
    global exp1, exp2, exp3
    tabla = []
    for n in nodos:
        fila = dict()
        fila["tamaño"] = n
        for t in tiempos:
            exp1 = tsms[(tsms.tiempos == t) & (tsms.nodos == n)][base]
            exp2 = gurobis[(gurobis.tiempos == t) & (gurobis.nodos == n)][base]
            fila[f"gurobi {t}"] = stats.wilcoxon(exp1, exp2, alternative = "less").pvalue
        tabla.append(fila)
    return pd.DataFrame(tabla)
    
tabla = wilcoxon2(gurobis, tsms)
tabla = tabla[sorted(tabla.columns)]

tabla2 = tabla.set_index("tamaño")
tabla2.columns = ["180s", "60s"]
tabla2 = tabla2[["60s", "180s"]].reset_index()
tabla2.columns = ["instance Size","60s", "180s"]
latex = tabla2.to_latex(
    index = False,
    column_format = "c ccc",
    multicolumn_format = "c",
    multirow = True, 
    caption =  "p-values of the Wilcoxon test performed over the experiments",
    label = "tab:wilcoxon",
    position = "H",
    na_rep = "-",
    escape = False,
    float_format = '{:.2E}'.format)

print(latex)
with open("latex/wilcoxon.tex", "a") as file:
    file.write(latex)

\begin{table}[H]
\caption{p-values of the Wilcoxon test performed over the experiments}
\label{tab:wilcoxon}
\begin{tabular}{c ccc}
\toprule
instance Size & 60s & 180s \\
\midrule
100 & 6.82E-06 & - \\
150 & 1.22E-07 & 4.37E-31 \\
200 & 1.45E-13 & 4.93E-33 \\
\bottomrule
\end{tabular}
\end{table}



In [36]:
tabla2

,instance Size,60s,180s
0,100,6.816269e-06,NaN
1,150,1.221897e-07,4.366749e-31
2,200,1.449382e-13,4.928053e-33


In [47]:
def wilcoxon3(tsms, versiones, base = "gap_best", nodos = [100, 150, 200]):
    global exp1, exp2, exp3
    tabla = []
    for n in nodos:
        fila = dict()
        fila["tamaño"] = n
        for v in [1,2,3,4,5]:
            version = f"3SM-exp{v}"
            exp1 = tsms[(tsms.tiempos == (60 if n == 100 else 180)) & (tsms.nodos == n)][base]
            exp2 = versiones[(versiones.tiempos == (60 if n == 100 else 180)) & (versiones.nodos == n) & (versiones.version == version)][base]
            fila[version] = stats.wilcoxon(exp1, exp2, alternative = "less").pvalue
        tabla.append(fila)
    return pd.DataFrame(tabla)
    
tabla = wilcoxon3(tsms, versiones)
tabla = tabla[sorted(tabla.columns)]

tabla3 = tabla.set_index("tamaño")
tabla3 = tabla3.reset_index()
tabla3.columns = ["instance Size","3SM-exp1","3SM-exp2","3SM-exp3","3SM-exp4","3SM-exp5"]
latex = tabla3.to_latex(
    index = False,
    column_format = "c ccc",
    multicolumn_format = "c",
    multirow = True, 
    caption =  "p-values of the Wilcoxon test performed over the experiments",
    label = "tab:wilcoxon",
    position = "H",
    na_rep = "-",
    escape = False,
    float_format = '{:.2E}'.format)

print(latex)
with open("latex/wilcoxon.tex", "a") as file:
    file.write(latex)

\begin{table}[H]
\caption{p-values of the Wilcoxon test performed over the experiments}
\label{tab:wilcoxon}
\begin{tabular}{c ccc}
\toprule
instance Size & 3SM-exp1 & 3SM-exp2 & 3SM-exp3 & 3SM-exp4 & 3SM-exp5 \\
\midrule
100 & 1.71E-43 & 1.24E-32 & 2.90E-21 & 7.30E-24 & 7.02E-01 \\
150 & 6.07E-51 & 2.41E-09 & 2.87E-01 & 1.85E-04 & 2.57E-01 \\
200 & 4.97E-51 & 5.98E-27 & 3.40E-10 & 3.14E-01 & 2.17E-04 \\
\bottomrule
\end{tabular}
\end{table}



In [48]:
tabla3

,instance Size,3SM-exp1,3SM-exp2,3SM-exp3,3SM-exp4,3SM-exp5
0,100,1.708345e-43,1.243847e-32,2.897779e-21,7.295124e-24,0.701942
1,150,6.069502e-51,2.412847e-09,2.873584e-01,1.854178e-04,0.257230
2,200,4.969159e-51,5.982358e-27,3.397582e-10,3.140950e-01,0.000217


# Save BKS

In [31]:
bks["name"] = bks["name"].str.upper()
bks["fullname"] = bks["name"] + "-" + bks["Q"].astype("str") + "-" + bks["nodos"].astype("str") 
dict_bks = dict()
for i, row in bks.iterrows():
    dict_bks[row["fullname"]] = row["bks"]

In [32]:
import json
with open('bks.json', 'w') as f:
    json.dump(dict_bks, f)